# Week 5 - Data Analysis Using NumPy and Pandas 2

## Pandas Basics

Pandas를 사용하기 위해서 다음과 같이 라이브러리를 import 한다. numpy도 주로 같이 사용하니 같이 import 한다.

In [ ]:
import pandas as pd
import numpy as np

Pandas는 `read_csv()`라는 CSV 파일을 읽어주는 함수를 제공한다.

(참고) Excel 파일을 읽는 함수도 제공한다. `ExcelFile(), read_excel()`
```
* df = pd.ExcelFile("dummydata.xlsx")
* df = pd.read_excel(open('your_xls_xlsx_filename','rb'), sheetname='Sheet 1')
```

In [ ]:
data = pd.read_csv("data/weather_year.csv")

In [ ]:
data
len(data)

columns 명령어는 데이터의 column을 보여준다. 

In [ ]:
data.columns

특정 column을 읽어오려면 다음과 같이 쓴다.

In [ ]:
data['EDT']

어레이를 이용하여 여러개의 column을 불러올 수 있다.

In [ ]:
data[['EDT', 'Max TemperatureF', 'Mean TemperatureF']]

`describe()` 명령어는 기초 통계값을 보여준다. 숫자에만 적용된다.

In [ ]:
data[['EDT', 'Max TemperatureF', 'Mean TemperatureF']].describe()

만약 column 이름에 공백이 없다면, **```data.column_name```** 형태로 표현 가능하다.

In [ ]:
data.EDT # only works when the column name has no space

`head()`는 다섯개의 샘플 데이터만 보여준다.

In [ ]:
data.head()

`data.head(num)`는 원하는 수 num 만큼의 데이터를 보여준다.

In [ ]:
data[['EDT', 'Max TemperatureF', 'Mean TemperatureF']].head(10)

`tail()` 함수는 `head()`와 반대로 맨 아래 다섯개의 데이터를 보여준다.

In [ ]:
data[['EDT', 'Max TemperatureF', 'Mean TemperatureF']].tail()

다시 columns 를 출력해보자.

In [ ]:
data.columns

columns는 단어 중간이나 앞부분에 공백이 있기도 하고 길어서 데이터 처리에 불편하다. columns를 다음과 같이 고쳐보자. column들의 이름이 바뀌었다.

In [ ]:
data.columns = ["date", "max_temp", "mean_temp", "min_temp", "max_dew",
                "mean_dew", "min_dew", "max_humidity", "mean_humidity",
                "min_humidity", "max_pressure", "mean_pressure",
                "min_pressure", "max_visibilty", "mean_visibility",
                "min_visibility", "max_wind", "mean_wind", "min_wind",
                "precipitation", "cloud_cover", "events", "wind_dir"]

In [ ]:
data.head()

In [ ]:
data.min_temp.head()

In [ ]:
data.min_temp.std()

Pandas는 자체적으로 matplotlib을 지원하고 있다. 따라서 간단하게 데이터의 그래프를 그릴 수 있다.

Jupyter Notebook에서 그래프를 보이게 하려면 다음과 같은 명령어를 입력한다.

`%matplotlib inline`

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt  

# 이 코드에서 실제로 plt 는 사용하지 않지만, 후에 matplotlob.pyplot를 이용해 그림을 그린다면 필요

In [ ]:
data.min_temp.hist()

In [ ]:
data.min_temp.plot()

In [ ]:
data.describe()

## Bulk Data Clean Up

이번에는 `apply()`함수를 사용하여 column 내의 데이터를 한꺼번에 변경하는 방법을 다룬다.

In [ ]:
data.date.head()

In [ ]:
first_date = data.date.values[0]
type(first_date)

지난 번에 사용했던 방법으로 str 오브젝트를 datetime 오브젝트로 변환해보자.

In [ ]:
# from datetime import datetime
# first_date = datetime.strptime(first_date, "%Y-%m-%d")

type(first_date)

`lambda`와 `apply()`를 사용해서 데이터의 형을 변환한다. `apply()`는 bulk operation을 가능하게 한다. 즉, column에 있는 모든 요소에 한꺼번에 적용한다.

In [ ]:
# (참고) lambda 의 사용법
function = lambda x: x*5
function(5)

In [ ]:
data.date = data.date.apply(lambda d: datetime.strptime(d, "%Y-%m-%d"))
data.date.head()

In [ ]:
data.head()

현재 우리 데이터의 각각의 row의 index는 `0 ~ len(data)`의 integer가 자동으로 지정되었다. 만약 데이터의 index를 다른 컬럼으로 (예를 들어 date) 지정하고 싶으면 index 명령어를 사용한다.

In [ ]:
data.index = data.date
data.head()

이제 데이터를 날짜를 이용해서 검색할 수 있다.

In [ ]:
data.loc[datetime(2012, 10, 18)]

불필요한 컬럼을 삭제하기 위해서는 drop() 명령어를 사용한다. 위의 데이터에서는 date를 index로 지정하였기 때문에 데이터 테이블 내의 date는 필요없다. (현재 중복되어 있다) 

다음과 같이 삭제할 수 있다.

In [ ]:
data = data.drop(["date"], axis=1)
data.columns

In [ ]:
data

In [ ]:
data.head()

axis=1 은 column에 적용됨을 의미한다. 0은 row에 1은 column에 적용된다고 생각하면 된다. 
http://stackoverflow.com/questions/25773245/ambiguity-in-pandas-dataframe-numpy-array-axis-definition

## Missing Values

데이터에 missing value 가 있다. missing value 를 처리해보자.

In [ ]:
empty = data.apply(lambda col: pd.isnull(col))
empty

In [ ]:
empty.events.head(10)

In [ ]:
data.events.head(10)

In [ ]:
len(empty[empty.events == True])

missing data를 처리하는 한가지 방법은 해당 데이터 포인트를 삭제하는 것. pandas에는 `dropna()`라는 함수가 있다. 이 함수를 사용하면 데이터가 입력되지 않은 row, 즉 위의 데이터에서 events가 NaN인 경우 모두 삭제를 한다. 그러나, 이렇게 삭제를 하게되면 전체 366개의 데이터 중 162개 밖에 남지 않는다. 따라서 이렇게 missing data가 많은 경우 삭제하는 것 보다는 다르게 처리하는 편이 낫다. 우리는 NaN 대신 빈 문자열을 채워 넣으려고 한다. 이때 사용하는 함수는 `fillna()`이다.

In [ ]:
data.events = data.events.fillna("")
data.events.head(10)

## Accessing Rows

In [ ]:
data.iloc[0]

In [ ]:
data.iloc[3:9]

In [ ]:
data.loc[datetime(2012, 3, 15)]

`iterrows()`함수는 데이터프레임의 row를 iterate하게 한다. (for loop를 이용하여 한 row 씩 처리하게 한다.)

In [ ]:
num_rain = 0
for idx, row in data.iterrows():
    if "Rain" in row["events"]:
        num_rain += 1

"Days with rain: {0}".format(num_rain)

## Filtering

Filtering은 데이터를 살펴보고 분석을 할 때 가장 좋은 방법이다. Filtering을 하기 위한 방법들을 살펴보자.

In [ ]:
freezing_days = data[data.max_temp <= 32]
freezing_days

In [ ]:
freezing_days[freezing_days.min_temp >= 20]

In [ ]:
data[(data.max_temp <= 32) & (data.min_temp >= 20)]

필터를 변수 형태로 만들어 놓고 사용하는 것도 물론 가능하다.

In [ ]:
# max_temp 가 32보다 작거나 같은 경우의 필터를 생성
temp_max = data.max_temp <= 32
temp_max

In [ ]:
data[temp_max]

In [ ]:
# max_temp 가 20보다 크거나 같은 경우의 필터를 생성
temp_min = data.min_temp >= 20
temp_min

In [ ]:
temp_min & temp_max

In [ ]:
data[temp_min & temp_max]

두개의 필터를 이용해서 새로운 필터를 만들어 둘 수 있다.

In [ ]:
temp_both = temp_min & temp_max
data[temp_both]

숫자가 아닌 경우는 filter를 이런 식으로 쓸 수 없다. 다음과 같은 코드는 data.events의 각 row를 iterate하며 Rain이 포함되어 있는지 여부를 판단할 것 같지만 그렇지 않다. 이 코드는 에러를 발생한다.

In [ ]:
data["Rain" in data.events]

이 경우, 다음과 같이 lambda 함수와 apply 함수를 사용해 filter를 만든다.

In [ ]:
data[data.events.apply(lambda e: "Rain" in e)]

## Grouping

apply()만큼 유용하게 쓸 수 있는 함수 중에 groupby()가 있다. 이 함수는 dataframe에서 같은 값을 갖는 데이터 포인트를 찾아서 묶어 준다.

예를 들어 cloud_cover 데이터를 추출하면 다음과 같이 0-8까지 값이 기록되어 있다.

In [ ]:
data.cloud_cover

In [ ]:
data.cloud_cover.unique()

In [ ]:
for d in data.groupby("cloud_cover"):
    print(d)

# 결과는 tuple 형태로 출력된다. 
# d[0] -> cloud_cover level
# d[1] -> 각 cloud_cover level이 포함된 data row
# 여기서 loop는 각 data row의 iteration을 가져오는게 아니라 group의 iteration을 가져온다.

출력되는 tuple 값을 이용해서 각 그룹의 평균온도(mean_temp)의 평균을 다음과 같이 구할 수 있다.

In [ ]:
cover_temps = {}
for cover, cover_data in data.groupby("cloud_cover"):
    cover_temps[cover] = cover_data.mean_temp.mean()
cover_temps

한개 이상의 컬럼을 이용해서 데이터를 groupby 할 수 있다. 이 경우, 두개의 컬럼의 값이 tuple 로 먼저 출력되고, 그 다음에 data row 가 출력된다.

```예) a = ((1,2),3)
a[0] => (1,2)
a[1] => 3
a[0][0] = 1
```

In [ ]:
for (cover, events), group_data in data.groupby(["cloud_cover", "events"]):
    print("Cover: {}, Events: {}, Count: {}".format(cover, events, len(group_data)))

## Adding New Columns

events 컬럼의 데이터를 살펴보자.

In [ ]:
data.events.unique()

여러 데이터가 있지만, 기본적으로는 rain, thunderstorm, fog, snow 의 네가지의 조합이다. 특정일에 비가 왔는지, 안개가 끼었는지 등을 확인하고 싶어서 새롭게 컬럼을 만들고자 한다. rain, thunderstorm, fog, snow의 컬럼을 만들고 True or False를 저장할 것이다. (One Hot Coding)
```
* Rain-Thunderstorm => rain:True, thunderstorm:True, fog:False, snow:False
* Fog-Rain-Thunderstorm => rain:False, thunderstorm:True, fog:True, snow:False
```

In [ ]:
for event_kind in ["Rain", "Thunderstorm", "Fog", "Snow"]:
    col_name = event_kind.lower()  # Turn "Rain" into "rain", etc.
    data[col_name] = data.events.apply(lambda e: event_kind in e)
data

In [ ]:
data.rain

In [ ]:
data.snow.sum()

In [ ]:
data[data.rain | data.snow]

In [ ]:
def toCel(degree):
    return (degree - 32) / 1.8
    
data["max_tempc"] = data.max_temp.apply(lambda e: toCel(e))
data["min_tempc"] = data.min_temp.apply(lambda e: toCel(e))
data["mean_tempc"] = data.mean_temp.apply(lambda e: toCel(e))
data

## Plotting

In [ ]:
data.mean_tempc.plot()

In [ ]:
data.mean_tempc.tail(10).plot(kind="bar", rot=15)

다음의 그래프는 여러개 데이터의 그래프를 그려준다.

In [ ]:
ax = data.max_temp.plot(title="Min and Max Temperatures")
data.min_temp.plot(style="red")
data.mean_temp.plot(style="yellow")
ax.set_ylabel("Temperature (F)")

## Export Data

`to_csv()` 함수는 데이터프레임을 CSV형태로 저장한다.

In [ ]:
data.to_csv("data/weather-mod.csv")

## 실습

### 실습 1
* 주어진 employment.csv파일은 header가 없는 파일이다. 불러온 후 column의 header를 추가한다.
* 국가명을 인덱스로 지정한다.
* 불필요한 컬럼이 있으면 삭제한다.
* 상위 5개를 출력한다.

In [ ]:
employment = pd.read_csv("data/gapminder/employment.csv", header=None)
employment.columns = ["country", "employment"]
employment.index = employment.country
employment = employment.drop(['country'], axis=1)
employment.head()

### 실습 2
* 나머지 2개의 파일은 비슷한 구조로 되어 있어 employment.csv와 유사한 방법으로 불러오게 된다. 함수를 만들고 나머지 파일을 불러오자.
* life_expectancy.csv
* gdp_per_capita.csv

In [ ]:
def read_gapminder_data(filename, colname):
    data = pd.read_csv("data/gapminder/{}".format(filename), header=None)
    data.columns = ["country", colname]
    data.index = data.country
    data = data.drop(['country'], axis=1)
    return data

In [ ]:
life_exp = read_gapminder_data("life_expectancy.csv", "life_exp")
life_exp.head()

In [ ]:
gdp = read_gapminder_data("gdp_per_capita.csv", "gdp")
gdp.head()

### 실습 3
* 세개의 데이터프레임을 하나로 합치자.
* 데이터프레임을 합칠 때는 merge, join 등의 개념이 사용된다.
* http://pandas.pydata.org/pandas-docs/stable/merging.html 를 참고하자.
* (힌트) concat 이 사용된다.

In [ ]:
frames = [employment, life_exp, gdp]
gm_data = pd.concat(frames, axis=1)
gm_data

소팅을 해보자. 소팅은 다음과 같이 한다.
* data.sort_values([colname1, colname2, ...], ascending=[True, False, ...])

In [ ]:
gm_data.sort_values(["gdp", "life_exp"], ascending=False)

### 실습 4
* 전체 데이터프레임의 기술통계값을 출력하고 그래프를 그려보자.
* employment와 life_exp의 histogram을 그려보자.

### 실습 5-1
* employment rate이 가장 높은 나라와 가장 낮은 나라의 이름과 값을 출력하자.
* 구글검색 혹은 레퍼런스를 통해 가장 높은 값과 낮은 값을 가진 인덱스를 출력하는 함수를 찾아 적용해보자.

(검색어 예) pandas dataframe max index

### 실습 5-2
* gdp 상위 10개 국가의 평균과 하위 10개 국가의 리스트와 값을 구하고 그리고 평균의 차이를 구해보자.
* `sort_values()` 사용

### 실습 5-3
* gpd 상위 10개 국가의 기대수명과 취업률 평균을 구하고 하위 10개 국가의 기대수명과 취업률 평균과 비교해 보자.

### 실습 6-1
* 각 국가의 초등학교 수료율을 기록한 두개의 파일을 읽어 하나의 데이터프레임을 만들어 보자.
* 남자의 초등학교 수료율: male_completion_rate.csv
* 여자의 초등학교 수료율: female_completion_rate.csv

### 실습 6-2
* 초등학교 수료율이 남성보다 여성이 더 높은 나라를 찾아보자.
* 초등학교 수료율이 여성보다 남성이 더 높은 나라를 찾아보자.

### 실습 6-3
* 남성과 여성 간의 초등학교 수료율 차이가 큰 상위 20개의 나라를 찾아보자. (Top10 vs Botton10)